In [3]:
import pandas as pd
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import preprocess_input
from collections import OrderedDict

In [13]:
import cv2
import os
import pandas as pd

# Load the CSV file
csv_path = 'radiological_gradings.csv'
radiological_data = pd.read_csv(csv_path)

# Initialize lists for each label
pfirrman_grades = []
modic_types = []
up_endplates = []
low_endplates = []
disc_herniation = []
disc_narrowing = []
disc_bulging = []
spondylolisthesis = []
image_list = []  # For storing image data
image_names = []

# Directory containing the disc images
image_dir = "split_discs"

# Loop through each image in the directory
for filename in os.listdir(image_dir):
    if filename.endswith(".png"):
        # Parse patient number and IVD label from the filename
        base_name = filename.replace(".png", "")
        
        # Split the filename and handle cases with additional parts
        parts = base_name.split("_")
        patient_id = int(parts[0])           # First part is always the patient ID
        scan_type = parts[-2]                # Second last part is the scan type (t1/t2)
        ivd_label = int(parts[-1].replace("disc", ""))  # Last part is the disc label
        
        # Find the corresponding row in the CSV
        row = radiological_data[(radiological_data['Patient'] == patient_id) &
                                (radiological_data['IVD label'] == ivd_label)]
        
        # Ensure the row exists and extract label values
        if not row.empty:
            # Append each label to its respective list
            modic_types.append(row['Modic'].values[0])
            up_endplates.append(row['UP endplate'].values[0])
            low_endplates.append(row['LOW endplate'].values[0])
            spondylolisthesis.append(row['Spondylolisthesis'].values[0])
            disc_herniation.append(row['Disc herniation'].values[0])
            disc_narrowing.append(row['Disc narrowing'].values[0])
            disc_bulging.append(row['Disc bulging'].values[0])
            pfirrman_grades.append(row['Pfirrman grade'].values[0])
            
            # Optionally load the image data if needed
            image = Image.open(os.path.join(image_dir, filename))
            # Resize image to 224x224 pixels
            image = image.resize((224,224))
            # Convert image to RGB
            image = image.convert("RGB")
            # Convert the image to numpy array
            image_array = np.array(image)
            # Normalize array
            image_array = preprocess_input(image_array)
            image_list.append(image_array)
            image_names.append(filename)

print("Labels have been mapped to each image and stored in the corresponding lists.")

Labels have been mapped to each image and stored in the corresponding lists.


In [34]:
image_list_np = np.array(image_list)
up_endplates_np = np.array(up_endplates)
low_endplates_np = np.array(low_endplates)
disc_herniation_np = np.array(disc_herniation)
disc_narrowing_np = np.array(disc_narrowing)
disc_bulging_np = np.array(disc_bulging)
spondylolisthesis_np = np.array(spondylolisthesis)

In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce dimensions
x = Dense(1024, activation='relu')(x)  # Fully connected layer
up_predictions = Dense(1, activation='sigmoid', name='up_output')(x)
low_predictions = Dense(1, activation='sigmoid', name='low_output')(x)
herniation_predictions = Dense(1, activation='sigmoid', name='herniation_output')(x)
narrowing_predictions = Dense(1, activation='sigmoid', name='narrowing_output')(x)
bulging_predictions = Dense(1, activation='sigmoid', name='bulging_output')(x)
spondy_predictions = Dense(1, activation='sigmoid', name='spondy_output')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=[up_predictions, low_predictions, herniation_predictions, narrowing_predictions, bulging_predictions, spondy_predictions])
model.compile(optimizer='adam', loss={
        'up_output': 'binary_crossentropy',
        'low_output': 'binary_crossentropy',
        'herniation_output': 'binary_crossentropy',
        'narrowing_output': 'binary_crossentropy',
        'bulging_output': 'binary_crossentropy',
        'spondy_output': 'binary_crossentropy'
    },          
    metrics={
        'up_output': 'accuracy',
        'low_output': 'accuracy',
        'herniation_output': 'accuracy',
        'narrowing_output': 'accuracy',
        'bulging_output': 'accuracy',
        'spondy_output': 'accuracy'
    }
)

In [8]:
history = model.fit(
    image_list_np,            # Input images
    [up_endplates_np, low_endplates_np, disc_herniation_np, disc_narrowing_np, disc_bulging_np, spondylolisthesis_np],
    batch_size=32,
    epochs=10,
    validation_split=0.2,     # Split 20% of data for validation
    verbose=1                 # Set to 1 to see training progress
)

Epoch 1/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - bulging_output_accuracy: 0.5317 - bulging_output_loss: 0.9068 - herniation_output_accuracy: 0.8200 - herniation_output_loss: 0.8007 - loss: 3.9465 - low_output_accuracy: 0.5882 - low_output_loss: 0.7198 - narrowing_output_accuracy: 0.6070 - narrowing_output_loss: 0.7393 - spondy_output_accuracy: 0.9927 - spondy_output_loss: 0.0810 - up_output_accuracy: 0.5919 - up_output_loss: 0.6983 - val_bulging_output_accuracy: 0.5047 - val_bulging_output_loss: 1.0445 - val_herniation_output_accuracy: 0.9486 - val_herniation_output_loss: 0.2044 - val_loss: 3.6598 - val_low_output_accuracy: 0.5327 - val_low_output_loss: 0.7143 - val_narrowing_output_accuracy: 0.5374 - val_narrowing_output_loss: 0.7209 - val_spondy_output_accuracy: 0.9766 - val_spondy_output_loss: 0.1500 - val_up_output_accuracy: 0.4533 - val_up_output_loss: 0.8128
Epoch 2/10
27/27 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - bulging_output_accuracy: 0.6989 - bulging_output_loss: 0.5984 - hern

In [32]:
test_image_path = 'split_discs/127_t2_SPACE_disc_3.png'

# Load, resize, and preprocess the test image
img = Image.open(test_image_path).convert('RGB')
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)

# Preprocess the image for ResNet50
img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img_array)
img_preprocessed = np.expand_dims(img_preprocessed, axis=0)  # Add batch dimension

# Run the prediction
predicted_probabilities = model.predict(img_preprocessed)
print(f"Probability of Up-Endplate: {predicted_probabilities[0][0][0]}")
print(f"Probability of Low-Endplate: {predicted_probabilities[1][0][0]}")
print(f"Probability of Disc-Herniation: {predicted_probabilities[2][0][0]}")
print(f"Probability of Disc-Narrowing: {predicted_probabilities[3][0][0]}")
print(f"Probability of Disc-Bulging: {predicted_probabilities[4][0][0]}")
print(f"Probability of Spondylolisthesis: {predicted_probabilities[5][0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Probability of Up-Endplate: 0.5408105254173279
Probability of Low-Endplate: 0.6300442218780518
Probability of Disc-Herniation: 0.009400335140526295
Probability of Disc-Narrowing: 0.6716626882553101
Probability of Disc-Bulging: 0.9579753279685974
Probability of Spondylolisthesis: 0.004607596434652805


In [31]:
idx = 142
print(f"  Image name: {image_names[idx]}")
print(f"  Pfirrman Grade: {pfirrman_grades[idx]}")
print(f"  Modic Type: {modic_types[idx]}")
print(f"  UP Endplate: {up_endplates[idx]}")
print(f"  LOW Endplate: {low_endplates[idx]}")
print(f"  Spondylolisthesis: {spondylolisthesis[idx]}")
print(f"  Disc Herniation: {disc_herniation[idx]}")
print(f"  Disc Narrowing: {disc_narrowing[idx]}")
print(f"  Disc Bulging: {disc_bulging[idx]}")

  Image name: 127_t2_SPACE_disc_3.png
  Pfirrman Grade: 4
  Modic Type: 2
  UP Endplate: 1
  LOW Endplate: 1
  Spondylolisthesis: 0
  Disc Herniation: 0
  Disc Narrowing: 1
  Disc Bulging: 1


In [33]:
model.save('binaryIndividualPredictions.keras')